In [7]:
def validate_drowsiness_dataset(base_path):
    """Pre-scan the drowsiness dataset to identify problematic files"""
    print("\n=== Validating Drowsiness Dataset ===")

    subjects = [d for d in os.listdir(base_path)
                if os.path.isdir(os.path.join(base_path, d)) and d.isdigit()]

    total_images = 0
    missing_images = 0
    corrupted_images = 0

    for subject in subjects:
        frames_dir = os.path.join(base_path, subject, "frames")
        csv_path = os.path.join(frames_dir, "labels.csv")

        if not os.path.exists(csv_path):
            print(f"  No labels.csv in {subject}")
            continue

        df = pd.read_csv(csv_path)

        for _, row in df.iterrows():
            total_images += 1
            img_path = os.path.join(frames_dir, row['filename'])

            if not os.path.exists(img_path):
                missing_images += 1
                print(f"  Missing: {img_path}")
            else:
                try:
                    with Image.open(img_path) as img:
                        img.verify()
                except Exception as e:
                    corrupted_images += 1
                    print(f"  Corrupted: {img_path}")

    print(f"\n  Dataset Statistics:")
    print(f"    Total images referenced: {total_images}")
    print(f"    Missing images: {missing_images}")
    print(f"    Corrupted images: {corrupted_images}")
    print(f"    Valid images: {total_images - missing_images - corrupted_images}")
    print(f"    Data integrity: {(1 - (missing_images + corrupted_images) / total_images) * 100:.2f}%")

    return total_images, missing_images, corrupted_images

# Call this before creating datasets (optional)
validate_drowsiness_dataset("/Users/shikharsrivastava/Desktop/Thesis/Thesis/MultiTask/Drowsiness/S5")


=== Validating Drowsiness Dataset ===
  Missing: /Users/shikharsrivastava/Desktop/Thesis/Thesis/MultiTask/Drowsiness/S5/05/frames/01409.jpg

  Dataset Statistics:
    Total images referenced: 81456
    Missing images: 1
    Corrupted images: 0
    Valid images: 81455
    Data integrity: 100.00%


(81456, 1, 0)

In [ ]:
import os, random, torch, glob
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import timm
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# ====== Original Single-Frame Datasets for Gaze and Face ======
class GazeDataset(Dataset):
    def __init__(self, image_dir, label_csv, transform=None):
        self.image_dir = image_dir
        self.labels_df = pd.read_csv(label_csv)
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        frame_index = self.labels_df.iloc[idx]['Frame Index']
        gaze_label = int(self.labels_df.iloc[idx]['Gaze Zone'])
        image_path = os.path.join(self.image_dir, f"frame_{int(frame_index):04d}.jpg")
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, gaze_label

class FaceRecognitionDataset(Dataset):
    def __init__(self, samples, transform):
        self.samples, self.transform = samples, transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# ====== Temporal Dataset for Drowsiness (Blinking) ======
# ====== Temporal Dataset for Drowsiness with Error Handling ======
class TemporalDrowsinessDataset(Dataset):
    """Temporal dataset for blinking detection with robust error handling"""
    def __init__(self, subject_dirs, transform=None, sequence_length=5, stride=2):
        """
        Args:
            subject_dirs: List of subject directories
            transform: Image transformations
            sequence_length: Number of frames in temporal window
            stride: Step size between sequences
        """
        self.transform = transform
        self.sequence_length = sequence_length
        self.stride = stride
        self.samples = []

        print(f"  Building temporal dataset with window={sequence_length}, stride={stride}")
        total_sequences_attempted = 0
        valid_sequences = 0
        skipped_sequences = 0

        for subject_dir in subject_dirs:
            frames_dir = os.path.join(subject_dir, "frames")
            csv_path = os.path.join(frames_dir, "labels.csv")

            if not os.path.exists(csv_path):
                print(f"    Warning: No labels.csv found in {frames_dir}")
                continue

            df = pd.read_csv(csv_path)
            df = df.sort_values('filename')

            # Use stride for overlapping windows
            for i in range(0, len(df) - sequence_length + 1, stride):
                total_sequences_attempted += 1
                seq_files = []
                seq_labels = []
                all_valid = True

                # Check if all images in the sequence exist and are valid
                for j in range(i, i + sequence_length):
                    img_path = os.path.join(frames_dir, df.iloc[j]['filename'])

                    # Check if file exists and can be opened
                    if os.path.exists(img_path):
                        try:
                            # Try to open the image to verify it's valid
                            with Image.open(img_path) as img:
                                img.verify()  # Verify it's a valid image
                            seq_files.append(img_path)
                            seq_labels.append(df.iloc[j]['label'])
                        except Exception as e:
                            # Image exists but is corrupted
                            all_valid = False
                            break
                    else:
                        # Image doesn't exist
                        all_valid = False
                        break

                if all_valid and len(seq_files) == sequence_length:
                    # Use majority vote for label
                    target_label = 1 if sum(seq_labels) > len(seq_labels) // 2 else 0
                    self.samples.append((seq_files, target_label, seq_labels))
                    valid_sequences += 1
                else:
                    skipped_sequences += 1

        print(f"  Dataset creation complete:")
        print(f"    - Total sequences attempted: {total_sequences_attempted}")
        print(f"    - Valid sequences created: {valid_sequences}")
        print(f"    - Sequences skipped (missing/corrupted frames): {skipped_sequences}")

        if len(self.samples) == 0:
            print(f"    WARNING: No valid sequences found! Check your data.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        seq_files, label, seq_labels = self.samples[idx]
        frames = []

        # Load images with additional error handling
        for img_path in seq_files:
            try:
                image = Image.open(img_path).convert("RGB")
                if self.transform:
                    image = self.transform(image)
                frames.append(image)
            except Exception as e:
                # If an image fails to load despite our checks,
                # create a black frame as fallback
                print(f"    Warning: Failed to load {img_path} during training, using black frame")
                if self.transform:
                    # Create a black image and transform it
                    black_image = Image.new('RGB', (256, 256), color='black')
                    image = self.transform(black_image)
                else:
                    # Create a tensor directly
                    image = torch.zeros(3, 256, 256)
                frames.append(image)

        # Stack frames into temporal sequence
        try:
            frames = torch.stack(frames, dim=0)  # [T, C, H, W]
        except Exception as e:
            print(f"    Error stacking frames: {e}")
            # Return a zero tensor as fallback
            frames = torch.zeros(self.sequence_length, 3, 256, 256)

        return frames, label

# ====== Mixed Multi-Task Dataset ======
class MixedMultiTaskDataset(Dataset):
    """Handles both single-frame and temporal data"""
    def __init__(self, datasets):
        self.data = []
        for task, ds in datasets.items():
            if ds is not None and len(ds) > 0:
                self.data += [(task, i) for i in range(len(ds))]
        self.datasets = datasets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        task, i = self.data[idx]
        x, y = self.datasets[task][i]
        return x, y, task

# ====== Hybrid Multi-Task Model ======
class HybridMultiTaskModel(nn.Module):
    def __init__(self, backbone_name='mobilevit_s', pretrained=True,
                 gaze_classes=9, drowsy_classes=2, face_classes=15,
                 drowsy_sequence_length=5, hidden_dim=256):
        super().__init__()

        # Shared spatial feature extractor
        self.backbone = timm.create_model(backbone_name, pretrained=pretrained, num_classes=0)
        feat_dim = self.backbone.num_features
        self.drowsy_sequence_length = drowsy_sequence_length

        # Task-specific normalization
        self.gaze_norm = nn.LayerNorm(feat_dim)
        self.drowsy_norm = nn.LayerNorm(feat_dim)
        self.face_norm = nn.LayerNorm(feat_dim)

        # Temporal processing ONLY for drowsiness
        # Lighter LSTM for faster training
        self.drowsy_lstm = nn.LSTM(
            feat_dim, hidden_dim // 2,  # Reduced hidden size
            num_layers=1,  # Reduced layers for speed
            batch_first=True,
            dropout=0.2,
            bidirectional=True
        )

        # Simpler attention for speed
        self.drowsy_attention = nn.MultiheadAttention(
            hidden_dim,  # Matches LSTM output
            num_heads=2,  # Reduced heads
            dropout=0.1
        )

        # Task-specific heads
        self.gaze_head = nn.Sequential(
            nn.Linear(feat_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, gaze_classes)
        )

        self.drowsy_head = nn.Sequential(
            nn.Linear(hidden_dim, 64),  # Smaller head
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, drowsy_classes)
        )

        self.face_head = nn.Sequential(
            nn.Linear(feat_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, face_classes)
        )

    def extract_features(self, x):
        """Extract features handling both single images and sequences"""
        if len(x.shape) == 4:  # Single image [B, C, H, W]
            features = self.backbone.forward_features(x)
            features = features.mean(dim=[2, 3])  # GAP
            return features
        else:  # Sequence [B, T, C, H, W]
            batch_size, seq_len, c, h, w = x.size()
            x_flat = x.view(batch_size * seq_len, c, h, w)
            features = self.backbone.forward_features(x_flat)
            features = features.mean(dim=[2, 3])  # GAP
            features = features.view(batch_size, seq_len, -1)
            return features

    def forward(self, x, task):
        if task == 'gaze':
            # Single-frame processing
            features = self.extract_features(x)
            features = self.gaze_norm(features)
            return self.gaze_head(features)

        elif task == 'drowsy':
            # Temporal processing for blinking detection
            features = self.extract_features(x)  # [B, T, feat_dim]
            features = self.drowsy_norm(features)

            # LSTM for temporal modeling
            lstm_out, _ = self.drowsy_lstm(features)

            # Simple temporal pooling (faster than attention)
            # You can switch between mean pooling and attention
            USE_ATTENTION = False  # Toggle for speed vs accuracy

            if USE_ATTENTION:
                lstm_out = lstm_out.transpose(0, 1)
                attn_out, _ = self.drowsy_attention(lstm_out, lstm_out, lstm_out)
                temporal_features = attn_out.mean(dim=0)  # [B, hidden_dim]
            else:
                # Simple mean pooling (faster)
                temporal_features = lstm_out.mean(dim=1)  # [B, hidden_dim]

            return self.drowsy_head(temporal_features)

        else:  # face
            # Single-frame processing
            features = self.extract_features(x)
            features = self.face_norm(features)
            return self.face_head(features)

# ====== PCGrad with Dynamic Weight Decay ======
class PCGradWithDynamicWD(torch.optim.Optimizer):
    def __init__(self, optimizer):
        self._optim = optimizer
        self.current_stage = 0

    def zero_grad(self):
        return self._optim.zero_grad()

    def step(self):
        return self._optim.step()

    def update_weight_decay(self, stage, task_probs):
        """Dynamically adjust weight decay based on training stage"""
        self.current_stage = stage

        # Define weight decay strategies for different stages
        wd_strategies = {
            0: {  # Stage 1: Single task (Gaze only)
                'backbone': 1e-4,
                'temporal': 1e-4,
                'heads': 1e-4
            },
            1: {  # Stage 2: Gaze + Drowsy
                'backbone': 5e-4,  # Increase to prevent overfitting
                'temporal': 1e-3,  # Higher for LSTM
                'heads': 5e-4
            },
            2: {  # Stage 3: All tasks
                'backbone': 1e-3,  # Further increase
                'temporal': 2e-3,  # Highest for temporal
                'heads': 1e-3
            }
        }

        wd_config = wd_strategies.get(stage, wd_strategies[2])

        # Apply to parameter groups
        if len(self._optim.param_groups) >= 3:
            self._optim.param_groups[0]['weight_decay'] = wd_config['backbone']
            self._optim.param_groups[1]['weight_decay'] = wd_config['temporal']
            self._optim.param_groups[2]['weight_decay'] = wd_config['heads']

        print(f"  Weight Decay Updated - Backbone: {wd_config['backbone']}, "
              f"Temporal: {wd_config['temporal']}, Heads: {wd_config['heads']}")

    def update_learning_rates(self, stage):
        """Adjust learning rates based on stage"""
        lr_strategies = {
            0: {'backbone': 1e-4, 'temporal': 1e-3, 'heads': 1e-3},
            1: {'backbone': 5e-5, 'temporal': 5e-4, 'heads': 5e-4},
            2: {'backbone': 1e-5, 'temporal': 1e-4, 'heads': 1e-4}
        }

        lr_config = lr_strategies.get(stage, lr_strategies[2])

        if len(self._optim.param_groups) >= 3:
            self._optim.param_groups[0]['lr'] = lr_config['backbone']
            self._optim.param_groups[1]['lr'] = lr_config['temporal']
            self._optim.param_groups[2]['lr'] = lr_config['heads']

        print(f"  Learning Rates Updated - Backbone: {lr_config['backbone']}, "
              f"Temporal: {lr_config['temporal']}, Heads: {lr_config['heads']}")

    def pc_backward(self, objectives):
        self._optim.zero_grad()

        backbone_params = self._optim.param_groups[0]['params']
        temporal_params = self._optim.param_groups[1]['params'] if len(self._optim.param_groups) > 1 else []

        grads = []
        for i, loss in enumerate(objectives):
            for p in backbone_params + temporal_params:
                if p.grad is not None:
                    p.grad = None

            loss.backward(retain_graph=(i < len(objectives) - 1))

            single = []
            for p in backbone_params + temporal_params:
                if p.grad is None:
                    single.append(torch.zeros_like(p))
                else:
                    single.append(p.grad.detach().clone())
            grads.append(single)

        proj = grads[0]
        for other in grads[1:]:
            for j, (g, g2) in enumerate(zip(proj, other)):
                dot = (g * g2).sum()
                if dot < 0:
                    denom = g2.pow(2).sum() + 1e-12
                    proj[j] = g - (dot / denom) * g2

        for p, g in zip(backbone_params + temporal_params, proj):
            p.grad = g

# ====== Create Optimizer ======
def create_hybrid_optimizer(model, lr_backbone=1e-4, lr_temporal=1e-3, lr_heads=1e-3):
    backbone_params = list(model.backbone.parameters())

    temporal_params = []
    temporal_modules = [model.drowsy_lstm, model.drowsy_attention]
    for module in temporal_modules:
        temporal_params.extend(list(module.parameters()))

    head_params = []
    heads = [model.gaze_head, model.drowsy_head, model.face_head]
    norms = [model.gaze_norm, model.drowsy_norm, model.face_norm]
    for module in heads + norms:
        head_params.extend(list(module.parameters()))

    base_optim = torch.optim.AdamW([
        {'params': backbone_params, 'lr': lr_backbone, 'weight_decay': 1e-4},
        {'params': temporal_params, 'lr': lr_temporal, 'weight_decay': 1e-4},
        {'params': head_params, 'lr': lr_heads, 'weight_decay':1e-4}
    ])

    return PCGradWithDynamicWD(base_optim)  # Fixed typo here

# ====== Training Function ======
def train_hybrid(model, optim, device, criterions, task_probs, loaders, steps_per_epoch, loss_scale=None):
    if loss_scale is None:
        loss_scale = {}

    model.train()
    iters = {t: iter(loaders[t]) for t in loaders if loaders[t] is not None}
    total_loss = 0.0
    task_losses = {t: 0.0 for t in loaders}
    counts = {t: 0 for t in loaders}
    corrects = {t: 0 for t in loaders}
    totals = {t: 0 for t in loaders}

    for _ in tqdm(range(steps_per_epoch), desc="Train"):
        objectives, batch_info = [], []
        active_tasks = [t for t, p in task_probs.items() if p > 0 and t in iters]
        random.shuffle(active_tasks)

        for t in active_tasks:
            try:
                x, y = next(iters[t])
            except StopIteration:
                iters[t] = iter(loaders[t])
                x, y = next(iters[t])

            x, y = x.to(device), y.to(device)
            out = model(x, t)
            loss = criterions[t](out, y)

            if t in loss_scale:
                loss = loss * float(loss_scale[t])

            objectives.append(loss)
            batch_info.append((t, out, y))

        if not objectives:
            continue

        optim.zero_grad()
        optim.pc_backward(objectives)
        optim.step()

        with torch.no_grad():
            step_loss = sum(l.item() for l in objectives) / len(objectives)
            total_loss += step_loss

            for (t, out, y), L in zip(batch_info, objectives):
                preds = out.argmax(1)
                corrects[t] += (preds == y).sum().item()
                totals[t] += y.numel()
                task_losses[t] += L.item() * y.size(0)
                counts[t] += y.size(0)

    avg_task_losses = {t: task_losses[t] / max(counts[t], 1) for t in task_losses}
    accs = {t: 100 * corrects[t] / max(totals[t], 1) for t in totals}

    return total_loss / steps_per_epoch, accs, avg_task_losses

# ====== Evaluation Function ======
def evaluate_hybrid(model, loader, device, criterions):
    model.eval()
    preds = {t: [] for t in ['gaze', 'drowsy', 'face']}
    targets = {t: [] for t in ['gaze', 'drowsy', 'face']}

    with torch.no_grad():
        for x, y, task in loader:
            x, y = x.to(device), y.to(device)
            for i, t in enumerate(task):
                out = model(x[i].unsqueeze(0), t)
                pred = out.argmax(dim=1).item()
                preds[t].append(pred)
                targets[t].append(y[i].item())

    return {t: accuracy_score(targets[t], preds[t]) * 100 if len(preds[t]) > 0 else 0 for t in preds}

# ====== Main Training Script ======
if __name__ == "__main__":
    import multiprocessing
    multiprocessing.set_start_method('spawn', force=True)

    # ====== WINDOW SIZE ANALYSIS ======
    print("\n" + "="*60)
    print("WINDOW SIZE EFFECTS ON TRAINING SPEED:")
    print("="*60)
    print("""
    Window Size | Memory Usage | Training Speed | Dataset Size | Accuracy
    ------------|--------------|----------------|--------------|----------
    3 frames    | Low (3x)     | Fast           | Large        | Lower
    5 frames    | Medium (5x)  | Moderate       | Medium       | Good
    7 frames    | High (7x)    | Slower         | Smaller      | Better
    9 frames    | V.High (9x)  | Slow           | Small        | Best*

    * Diminishing returns after 7-9 frames for blinking detection

    Trade-offs:
    - Larger window = More temporal context = Better accuracy
    - Larger window = More memory = Smaller batch size = Slower training
    - Larger window = Fewer sequences (with same stride) = Less data variety

    Recommended: 5-7 frames for blinking (captures full blink cycle ~200-300ms)
    """)
    print("="*60 + "\n")

    # Configuration
    DROWSY_WINDOW_SIZE = 5  # Optimal balance (you can experiment: 3, 5, 7, 9)
    DROWSY_STRIDE = 2  # Overlap between windows (1=max overlap, window_size=no overlap)

    output_dir = "hybrid_training_results"
    plots_dir = os.path.join(output_dir, "plots")
    os.makedirs(plots_dir, exist_ok=True)

    # Data augmentation
    transform = transforms.Compose([
        transforms.RandomResizedCrop(256, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.25, scale=(0.02, 0.15))
    ])

    # === SUBJECT-WISE SPLITS FOR ALL DATASETS ===

    # === Gaze Dataset (Subject-wise split) ===
    print("\n=== Setting up Gaze Dataset ===")
    gaze_base = "/Users/shikharsrivastava/Desktop/Thesis/Thesis/MultiTask/Gaze/S6_face_RGB"
    gaze_subjects = list(range(1, 16))
    random.seed(42)
    random.shuffle(gaze_subjects)

    # 70-15-15 split
    n_train_gaze = int(len(gaze_subjects) * 0.7)
    n_val_gaze = int(len(gaze_subjects) * 0.15)

    train_gaze_subjects = gaze_subjects[:n_train_gaze]
    val_gaze_subjects = gaze_subjects[n_train_gaze:n_train_gaze + n_val_gaze]
    test_gaze_subjects = gaze_subjects[n_train_gaze + n_val_gaze:]

    print(f"Gaze subjects - Train: {train_gaze_subjects}, Val: {val_gaze_subjects}, Test: {test_gaze_subjects}")

    def build_gaze(subjects):
        datasets = []
        for s in subjects:
            img_dir = os.path.join(gaze_base, str(s), "Frames_RGB_Valid")
            csv_path = os.path.join(gaze_base, str(s), f"Valid_gaze_label_{s}.csv")
            if os.path.exists(img_dir) and os.path.exists(csv_path):
                datasets.append(GazeDataset(img_dir, csv_path, transform))
        return ConcatDataset(datasets) if datasets else None

    train_gaze = build_gaze(train_gaze_subjects)
    val_gaze = build_gaze(val_gaze_subjects)  # Fixed typo here
    test_gaze = build_gaze(test_gaze_subjects)
    print(f"Gaze data built - Train: {len(train_gaze) if train_gaze else 0}, "
          f"Val: {len(val_gaze) if val_gaze else 0}, "
          f"Test: {len(test_gaze) if test_gaze else 0}")

    # === Drowsiness Dataset (Subject-wise split) ===
    print("\n=== Setting up Drowsiness Dataset ===")
    drowsy_base = "/Users/shikharsrivastava/Desktop/Thesis/Thesis/MultiTask/Drowsiness/S5"

    # Get all subject directories (numbered folders)
    drowsy_subjects = [
        d for d in os.listdir(drowsy_base)
        if os.path.isdir(os.path.join(drowsy_base, d)) and d.isdigit()
    ]
    drowsy_subjects = sorted(drowsy_subjects, key=int)  # Sort numerically

    print(f"Found {len(drowsy_subjects)} drowsiness subjects: {drowsy_subjects}")

    # Subject-wise split (70-15-15)
    random.seed(42)
    random.shuffle(drowsy_subjects)

    n_train_drowsy = int(len(drowsy_subjects) * 0.7)
    n_val_drowsy = int(len(drowsy_subjects) * 0.15)

    train_drowsy_subjects = drowsy_subjects[:n_train_drowsy]
    val_drowsy_subjects = drowsy_subjects[n_train_drowsy:n_train_drowsy + n_val_drowsy]
    test_drowsy_subjects = drowsy_subjects[n_train_drowsy + n_val_drowsy:]

    print(f"Drowsy subjects - Train: {train_drowsy_subjects}, Val: {val_drowsy_subjects}, Test: {test_drowsy_subjects}")

    # Convert subject IDs to full paths
    train_drowsy_dirs = [os.path.join(drowsy_base, s) for s in train_drowsy_subjects]
    val_drowsy_dirs = [os.path.join(drowsy_base, s) for s in val_drowsy_subjects]
    test_drowsy_dirs = [os.path.join(drowsy_base, s) for s in test_drowsy_subjects]

    # Create temporal datasets
    print(f"\nCreating temporal drowsiness datasets (window={DROWSY_WINDOW_SIZE}, stride={DROWSY_STRIDE})...")

    train_drowsy = TemporalDrowsinessDataset(
        train_drowsy_dirs, transform, DROWSY_WINDOW_SIZE, DROWSY_STRIDE
    )
    val_drowsy = TemporalDrowsinessDataset(
        val_drowsy_dirs, transform, DROWSY_WINDOW_SIZE, DROWSY_STRIDE
    )
    test_drowsy = TemporalDrowsinessDataset(
        test_drowsy_dirs, transform, DROWSY_WINDOW_SIZE, DROWSY_STRIDE
    )

    print(f"Drowsiness data built - Train: {len(train_drowsy)}, Val: {len(val_drowsy)}, Test: {len(test_drowsy)}")

    # === Face Recognition Dataset (Subject-wise split) ===
    print("\n=== Setting up Face Recognition Dataset ===")
    face_root = "/Users/shikharsrivastava/Desktop/Thesis/Thesis/MultiTask/Gaze/S6_face_RGB"

    # Same subjects as gaze (they're the same dataset)
    def build_face_datasets(subjects, transform):
        samples = []
        pid_list = [f"{i:02d}" for i in subjects]
        id_to_label = {pid: i for i, pid in enumerate([f"{j:02d}" for j in range(1, 16)])}

        for pid in pid_list:
            label = id_to_label[pid]
            img_dir = os.path.join(face_root, pid, "Frames_RGB_Valid")
            if not os.path.exists(img_dir):
                continue

            img_paths = [
                os.path.join(img_dir, f) for f in os.listdir(img_dir)
                if f.endswith(".jpg")
            ]
            img_paths.sort()
            samples.extend([(p, label) for p in img_paths])

        return FaceRecognitionDataset(samples, transform) if samples else None

    train_face = build_face_datasets(train_gaze_subjects, transform)
    val_face = build_face_datasets(val_gaze_subjects, transform)
    test_face = build_face_datasets(test_gaze_subjects, transform)

    print(f"Face data built - Train: {len(train_face) if train_face else 0}, "
          f"Val: {len(val_face) if val_face else 0}, "
          f"Test: {len(test_face) if test_face else 0}")

    # === Create DataLoaders ===
    # Adjust batch size based on window size
    base_batch_size = 32
    temporal_batch_size = max(4, base_batch_size // DROWSY_WINDOW_SIZE)  # Scale down for memory

    print(f"\nBatch sizes - Regular: {base_batch_size}, Temporal: {temporal_batch_size}")

    loaders = {}
    if train_gaze and len(train_gaze) > 0:
        loaders['gaze'] = DataLoader(train_gaze, batch_size=base_batch_size, shuffle=True, num_workers=0)
    if train_drowsy and len(train_drowsy) > 0:
        loaders['drowsy'] = DataLoader(train_drowsy, batch_size=temporal_batch_size, shuffle=True, num_workers=0)
    if train_face and len(train_face) > 0:
        loaders['face'] = DataLoader(train_face, batch_size=base_batch_size, shuffle=True, num_workers=0)

    # Validation and test loaders
    val_datasets = {}
    test_datasets = {}

    if val_gaze and len(val_gaze) > 0:
        val_datasets['gaze'] = val_gaze
    if val_drowsy and len(val_drowsy) > 0:
        val_datasets['drowsy'] = val_drowsy
    if val_face and len(val_face) > 0:
        val_datasets['face'] = val_face

    if test_gaze and len(test_gaze) > 0:
        test_datasets['gaze'] = test_gaze
    if test_drowsy and len(test_drowsy) > 0:
        test_datasets['drowsy'] = test_drowsy
    if test_face and len(test_face) > 0:
        test_datasets['face'] = test_face

    val_loader = DataLoader(
        MixedMultiTaskDataset(val_datasets),
        batch_size=1  # Process one at a time due to mixed dimensions
    ) if val_datasets else None

    test_loader = DataLoader(
        MixedMultiTaskDataset(test_datasets),
        batch_size=1
    ) if test_datasets else None

    # === Model Setup ===
    device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    print(f"\nUsing device: {device}")

    model = HybridMultiTaskModel(
        backbone_name='mobilevit_s',
        pretrained=True,
        gaze_classes=9,
        drowsy_classes=2,  # Binary: blinking or not
        face_classes=15,
        drowsy_sequence_length=DROWSY_WINDOW_SIZE
    ).to(device)

    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")

    criterions = {
        'gaze': nn.CrossEntropyLoss(label_smoothing=0.1),
        'drowsy': nn.CrossEntropyLoss(label_smoothing=0.05, weight=torch.tensor([1.0, 2.0]).to(device)),  # Weight blinking class higher
        'face': nn.CrossEntropyLoss(label_smoothing=0.1),
    }

    # Create optimizer
    optimizer = create_hybrid_optimizer(model)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer._optim, T_0=10, T_mult=2, eta_min=1e-6
    )

    # === Training Loop ===
    steps_per_epoch = max(len(loaders[t]) for t in loaders) if loaders else 100
    train_losses, val_losses = [], []
    gaze_train_accs, gaze_val_accs = [], []
    drowsy_train_accs, drowsy_val_accs = [], []
    face_train_accs, face_val_accs = [], []
    num_epochs = 20
    gaze_task_losses_list, drowsy_task_losses_list, face_task_losses_list = [], [], []
    best_val_loss = float('inf')
    loss_scale = {'face': 0.3, 'drowsy': 1.5}  # Boost drowsy loss

    print(f"\nStarting training for {num_epochs} epochs...")
    print(f"Steps per epoch: {steps_per_epoch}")

    for epoch in range(num_epochs):
        # === Staged Training with Dynamic Weight Decay ===
        if epoch < 8:
            # Stage 1: Focus on gaze (single-frame)
            stage = 0
            task_probs = {'gaze': 1, 'drowsy': 0.0, 'face': 0.0}
            if epoch == 0:
                optimizer.update_weight_decay(stage, task_probs)
                optimizer.update_learning_rates(stage)

        elif epoch < 18:
            # Stage 2: Add temporal drowsiness
            stage = 1
            task_probs = {'gaze': 0.7, 'drowsy': 0.3, 'face': 0.0}
            if epoch == 8:
                optimizer.update_weight_decay(stage, task_probs)
                optimizer.update_learning_rates(stage)

        else:
            # Stage 3: All tasks
            stage = 2
            task_probs = {'gaze': 0.65, 'drowsy': 0.25, 'face': 0.1}
            if epoch == 18:
                optimizer.update_weight_decay(stage, task_probs)
                optimizer.update_learning_rates(stage)

        print(f"\nEpoch {epoch+1}/{num_epochs} | Stage {stage+1} | task_probs={task_probs}")

        # Train
        loss, accs, task_losses = train_hybrid(
            model, optimizer, device, criterions,
            task_probs, loaders, steps_per_epoch, loss_scale
        )

        gaze_task_losses_list.append(task_losses.get('gaze', 0))
        drowsy_task_losses_list.append(task_losses.get('drowsy', 0))
        face_task_losses_list.append(task_losses.get('face', 0))

        # Evaluate
        if val_loader:
            val_accs = evaluate_hybrid(model, val_loader, device, criterions)

            # Compute validation loss
            def compute_val_loss_hybrid(model, loader, device, criterions):
                model.eval()
                total_loss, total_samples = 0, 0
                with torch.no_grad():
                    for x, y, task in loader:
                        x, y = x.to(device), y.to(device)
                        for i, t in enumerate(task):
                            out = model(x[i].unsqueeze(0), t)
                            loss = criterions[t](out, y[i].unsqueeze(0))
                            total_loss += loss.item()
                            total_samples += 1
                return total_loss / max(total_samples, 1)

            val_loss = compute_val_loss_hybrid(model, val_loader, device, criterions)
        else:
            val_accs = {'gaze': 0, 'drowsy': 0, 'face': 0}
            val_loss = float('inf')

        scheduler.step()

        print(f"Epoch {epoch+1} TrainLoss {loss:.4f} "
              f"(Gaze {task_losses.get('gaze', 0):.4f} Drowsy {task_losses.get('drowsy', 0):.4f} Face {task_losses.get('face', 0):.4f}) "
              f"ValLoss {val_loss:.4f} "
              f"ValAcc Gaze {val_accs['gaze']:.2f}% Drowsy {val_accs['drowsy']:.2f}% Face {val_accs['face']:.2f}%")

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_hybrid_model.pth")
            print(f"*** Best model updated at Epoch {epoch+1} with Val Loss {val_loss:.4f} ***")

        # Store metrics
        train_losses.append(loss)
        val_losses.append(val_loss)
        gaze_train_accs.append(accs.get('gaze', 0))
        gaze_val_accs.append(val_accs['gaze'])
        drowsy_train_accs.append(accs.get('drowsy', 0))
        drowsy_val_accs.append(val_accs['drowsy'])
        face_train_accs.append(accs.get('face', 0))
        face_val_accs.append(val_accs['face'])

    # === Test with best model ===
    print("\nTesting with best hybrid model...")
    if os.path.exists("best_hybrid_model.pth"):
        model.load_state_dict(torch.load("best_hybrid_model.pth"))

    if test_loader:
        test_accs = evaluate_hybrid(model, test_loader, device, criterions)
    else:
        test_accs = {'gaze': 0, 'drowsy': 0, 'face': 0}

    print(f"\n{'=' * 60}")
    print("🎉 HYBRID TRAINING COMPLETE! 🎉")
    print(f"{'=' * 60}")
    print(f"Configuration:")
    print(f"  Temporal Window Size: {DROWSY_WINDOW_SIZE} frames")
    print(f"  Temporal Stride: {DROWSY_STRIDE}")
    print(f"  Subject-wise splits for all tasks")
    print(f"\nTest Results:")
    print(f"  Gaze Detection (Single-frame):     {test_accs['gaze']:.2f}%")
    print(f"  Blinking Detection (Temporal):     {test_accs['drowsy']:.2f}%")
    print(f"  Face Recognition (Single-frame):   {test_accs['face']:.2f}%")
    print(f"\nBest model saved: best_hybrid_model.pth")
    print(f"{'=' * 60}")


WINDOW SIZE EFFECTS ON TRAINING SPEED:

    Window Size | Memory Usage | Training Speed | Dataset Size | Accuracy
    ------------|--------------|----------------|--------------|----------
    3 frames    | Low (3x)     | Fast           | Large        | Lower
    5 frames    | Medium (5x)  | Moderate       | Medium       | Good
    7 frames    | High (7x)    | Slower         | Smaller      | Better
    9 frames    | V.High (9x)  | Slow           | Small        | Best*

    * Diminishing returns after 7-9 frames for blinking detection

    Trade-offs:
    - Larger window = More temporal context = Better accuracy
    - Larger window = More memory = Smaller batch size = Slower training
    - Larger window = Fewer sequences (with same stride) = Less data variety

    Recommended: 5-7 frames for blinking (captures full blink cycle ~200-300ms)
    


=== Setting up Gaze Dataset ===
Gaze subjects - Train: [9, 14, 8, 7, 15, 13, 6, 3, 10, 4], Val: [5, 12], Test: [1, 2, 11]
Gaze data built - Tr

/Users/shikharsrivastava/Documents/envs/thesis/lib/python3.13/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


Total parameters: 6,344,058
Trainable parameters: 6,344,058

Starting training for 20 epochs...
Steps per epoch: 4537
  Weight Decay Updated - Backbone: 0.0001, Temporal: 0.0001, Heads: 0.0001
  Learning Rates Updated - Backbone: 0.0001, Temporal: 0.001, Heads: 0.001

Epoch 1/20 | Stage 1 | task_probs={'gaze': 1, 'drowsy': 0.0, 'face': 0.0}


Train:  22%|██▏       | 1009/4537 [07:37<26:55,  2.18it/s]